In [1]:
from qiskit import *
import math
simulator = Aer.get_backend('qasm_simulator')

In [2]:
def quantum_value(ctrl, value, name):
    """
    Encodes the bit values of the number in the circuit
    returns: encoded circuit
    """
    
    # make a circuit 
    circ = QuantumCircuit(len(value), name = name)
    
    # encode the value 
    for i in range(len(value)):
        if value[i] == '1':
            circ.x(i)
            
    circ = circ.control(num_ctrl_qubits = len(ctrl), ctrl_state = ctrl)
    
    return circ

In [3]:
def qc_vector(n, size_index, size_value, input_vector, first=True):
    """
    Creates the register for the list of numbers. Encodes 
    the index and the value of the number in the register and returns it.
    returns: QuantumCircuit object
    """
 
    index_register = QuantumRegister(size = size_index, name = "indices")
    value_register = QuantumRegister(size = size_value, name = "values")
    
    qcirc = QuantumCircuit(index_register, value_register)
    
    # for first number H needs to be placed first before encoding
    if first:
        qcirc.h(index_register)
        
    # start encoding of the bits 
    for i in range(n):
        # get the ctrl state 
        
        ctrl_state = bin(i)[2:].zfill(size_index)
        
        value = bin(input_vector[i])[2:].zfill(size_value)
        
        qcirc = qcirc.compose(quantum_value(ctrl_state, value, f'arr_{i}'))
    
    # for the second number H needs to be placed after encoding
    if not first:
        qcirc.h(index_register)
    
    return qcirc

In [4]:
def equality_oracle(qubit_number, qram1, qram2):
    """
    Creates the oracle by putting both the qrams together in the circuit
    returns: QuantumCircuit object
    """
    
    qc = QuantumCircuit(qubit_number, name = 'oracle')
    
    # make the oracle 
    qc.compose(qram1, inplace = True)
    qc.barrier()
    qc.compose(qram2, inplace = True)
    
    
    return qc 

In [5]:
def quantum_isEqual(value1, value2):
    """
    This function takes 2 numbers as an input. Creates a Circuit to compare both the numbers.
    returns: '1' if the numbers are equal to each other otherwise '0'
    """
    
    input_vector1 = [int(x) for x in str(value1)]
    input_vector2 = [int(x) for x in str(value2)]
    
    n = len(input_vector1) if len(input_vector1) > len(input_vector2) else len(input_vector2)
    m = max(input_vector1) if max(input_vector1) > max(input_vector2) else max(input_vector2)
    
    # fill 0's to match the number of digits in both inputs like [6] --> [0, 6]
    input_vector1 = [0]*(n - len(input_vector1)) + input_vector1
    input_vector2 = [0]*(n - len(input_vector2)) + input_vector2
    
    if n == 1:
        size_index = 1
    else:
        size_index = math.ceil(math.log2(n))
    size_value = math.floor(math.log2(m)) + 1

    qram1 = qc_vector(len(input_vector1), size_index, size_value, input_vector1, first=True)
    qram2 = qc_vector(len(input_vector2), size_index, size_value, input_vector2, first=False)
    
    num_qubits = qram1.num_qubits 
    
    final_circuit = QuantumCircuit(num_qubits + 1,1)
    
    # build oracle 
    oracle = equality_oracle(num_qubits, qram1, qram2)
    
    # add oracle
    final_circuit.compose(oracle, inplace = True )
    
    # build and add the mcx gate 
    x_gate = QuantumCircuit(1, name = 'x')
    x_gate.x(0)
    zero_mcx_gate = x_gate.control(num_ctrl_qubits=num_qubits, ctrl_state = 0)
    
    # compose the gate
    final_circuit.compose(zero_mcx_gate, inplace = True)
    
    # measurement of flag qubit 
    final_circuit.measure(num_qubits,0)
    results = execute(final_circuit, backend = simulator, shots = 2**12).result().get_counts()
    
    # all the counts were 1, means all zero state on the first
    if '0' not in results.keys():
        return 1
    else:
        return 0

This function takes 4 sides as an input and compares them with each other. In total, there will be 6 comparisons:
<br>1. a compared to b, c, d
<br>2. b compared to c, d
<br>3. c compared to d

And for a rectangle exactly 2 sides will match. In another case, if it is a Square, all 6 comparisons will return True

In [6]:
def is_rectangle(a, b, c, d):
    """
    Function takes 4 numbers and verifies if those numbers can be the 
    sides of a rectangle. 
    parameters: a, b, c, d: 4 sides
    returns: '1' if sides represents a rectangle otherwise '0'
    """
    
    equal_sides = 0
    sides = [a, b, c, d]
    
    for i, side1 in enumerate(sides):
        for side2 in sides[i+1:len(sides)]:
            
            # compare 2 sides
            result = quantum_isEqual(side1, side2)
            
            if result:
                equal_sides += 1
    
    if equal_sides == 2 or equal_sides == 6:
        return '1'
    else:
        return '0'

In [7]:
is_rectangle(5, 6, 6, 5)

'1'

In [8]:
def is_rectangle_classical(a, b, c, d):
    """
    Function takes 4 numbers and verifies if those numbers can be the 
    sides of a rectangle. 
    parameters: a, b, c, d: 4 sides
    returns: '1' if sides represents a rectangle otherwise '0'
    """
    
    equal_sides = 0
    sides = [a, b, c, d]
    
    for i, side1 in enumerate(sides):
        for side2 in sides[i+1:len(sides)]:
            if side1 == side2:
                equal_sides += 1
    
    if equal_sides == 2 or equal_sides == 6:
        return '1'
    else:
        return '0'

In [9]:
import random

def test(num_test):
    
    success = 0
    failure = 0

    for i in range(num_test):
        sides = list()
        for i in range(4):
            n = random.randint(1,30)
            sides.append(n)

        quantum_result = is_rectangle(sides[0], sides[1], sides[2], sides[3])
        classical_result = is_rectangle_classical(sides[0], sides[1], sides[2], sides[3])

        if quantum_result == classical_result:
            success += 1
        else:
            failure += 1
        
    return {"Success": success, "Failure": failure}

In [10]:
test(1000)

{'Success': 1000, 'Failure': 0}